In [1]:
import sys
import os
from pathlib import Path

# Add parent directory to Python path
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent
sys.path.append(str(project_dir))


# notebooks/02_model_training.ipynb

import pandas as pd
import numpy as np
from src.data_processor import DataProcessor
from src.models.expenditure_predictor import ExpenditurePredictor
import plotly.graph_objects as go

# Prepare data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')

# Train separate models for each household
household_models = {}
household_metrics = {}

for household in df['HouseholdID'].unique():
    print(f"\nTraining model for {household}")
    
    # Filter data for current household
    household_df = df[df['HouseholdID'] == household].copy()
    X, y = processor.prepare_training_data(household_df)
    
    # Split data
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Train model
    predictor = ExpenditurePredictor()
    history = predictor.train(X_train, y_train)
    
    # Evaluate model
    y_pred = predictor.predict(X_test)
    mse = np.mean((y_test - y_pred.flatten()) ** 2)
    
    # Store model and metrics
    household_models[household] = predictor
    household_metrics[household] = {
        'mse': mse,
        'history': history
    }
    
    print(f"MSE for {household}: {mse:.4f}")


# Save models for each household
for household in household_models:
    model_path = f'../src/models/saved/model_{household}.keras'
    household_models[household].save(model_path)
    print(f"Saved model for {household} at {model_path}")

# Visualize training history for each household
fig = go.Figure()
for household, metrics in household_metrics.items():
    history = metrics['history']
    fig.add_trace(go.Scatter(
        y=history.history['loss'],
        name=f'{household} Training Loss',
        mode='lines'
    ))
    fig.add_trace(go.Scatter(
        y=history.history['val_loss'],
        name=f'{household} Validation Loss',
        mode='lines',
        line=dict(dash='dash')
    ))

fig.update_layout(
    title='Training History by Household',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    hovermode='x unified'
)
fig.show()

# Compare predictions vs actual for each household
for household in household_models:
    household_df = df[df['HouseholdID'] == household].copy()
    X, y = processor.prepare_training_data(household_df)
    
    # Get predictions
    y_pred = household_models[household].predict(X)
    
    # Plot actual vs predicted
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=y,
        name='Actual',
        mode='lines'
    ))
    fig.add_trace(go.Scatter(
        y=y_pred.flatten(),
        name='Predicted',
        mode='lines'
    ))
    
    fig.update_layout(
        title=f'Actual vs Predicted Expenditure - {household}',
        xaxis_title='Time',
        yaxis_title='Expenditure ($)',
        hovermode='x unified'
    )
    fig.show()



Training model for H1
Epoch 1/100


c:\Users\utkar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 281ms/step - loss: 0.2878 - val_loss: 0.1478
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.2258 - val_loss: 0.1063
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.1593 - val_loss: 0.0663
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.1099 - val_loss: 0.0316
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0595 - val_loss: 0.0099
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0212 - val_loss: 0.0133
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0107 - val_loss: 0.0397
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0247 - val_loss: 0.0438
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0222 - val_loss: 0.0271
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0155 - val_loss: 0.0135
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0087 - val_loss: 0.0088
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0097 - val_loss: 0.0078


c:\Users\utkar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 269ms/step - loss: 0.5178 - val_loss: 0.2827
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.4441 - val_loss: 0.2453
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.3762 - val_loss: 0.2020
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.3217 - val_loss: 0.1548
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.2548 - val_loss: 0.1052
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.1765 - val_loss: 0.0578
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0938 - val_loss: 0.0244
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0420 - val_loss: 0.0278
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0171 - val_loss: 0.0633
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0366 - val_loss: 0.0815
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0471 - val_loss: 0.0639
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0273 - val_loss: 0.0388


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step 
